## Get the Workspace

In [7]:
from azureml.core.workspace import Workspace

workspace = Workspace.from_config()

## Set the Compute Target

In [8]:
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

cluster_name = 'aml-labs-01'
compute_target = ComputeTarget(workspace=workspace, name=cluster_name)

## Create an experiment

In [9]:
from azureml.core import Experiment

experiment_name = 'churn-experiment'
experiment = Experiment(workspace=workspace, name=experiment_name)

## Create Environment

In [10]:
import os
import shutil

project_folder = './xgboost-churn'
os.makedirs(project_folder, exist_ok=True)

# Copy the script to the project folder
shutil.copy('src/train.py', project_folder)

'./xgboost-churn/train.py'

In [11]:
from azureml.core import ScriptRunConfig, Environment
from azureml.core.conda_dependencies import CondaDependencies

xgboost_env = Environment(workspace=workspace, name="XGBoost-Env")
conda_dep = CondaDependencies()

conda_dep.add_pip_package("azureml-dataprep[pandas]")
conda_dep.add_pip_package("azureml-defaults")
conda_dep.add_pip_package("xgboost==0.90")
conda_dep.add_pip_package("scikit-learn==0.22.2.post1")
conda_dep.add_pip_package("numpy==1.18.5")

# Adds dependencies to PythonSection of myenv
xgboost_env.python.conda_dependencies=conda_dep

src = ScriptRunConfig(source_directory=project_folder,
                      script='train.py',
                      arguments=['--max_depth', 64, 
                                 '--learning_rate', 0.2260, 
                                 '--reg_alpha', 0.01, 
                                 '--reg_lambda', 0.348, 
                                 '--min_child_weight', 19.22],
                      compute_target=compute_target,
                      environment=xgboost_env)

In [18]:
from azureml.train.hyperdrive import HyperDriveConfig
from azureml.train.hyperdrive import RandomParameterSampling, BanditPolicy, quniform, loguniform, uniform, PrimaryMetricGoal

SEED = 2020

param_sampling = RandomParameterSampling({
  '--max_depth': quniform(4, 100, 1),
  '--learning_rate': loguniform(-3, 0),
  '--reg_alpha': loguniform(-5, -1),
  '--reg_lambda': loguniform(-6, -1),
  '--min_child_weight': loguniform(-1, 3),
})

early_termination_policy = BanditPolicy(slack_factor=0.15, evaluation_interval=1, delay_evaluation=10)

hd_config = HyperDriveConfig(run_config=src,
                             hyperparameter_sampling=param_sampling,
                             policy=early_termination_policy,
                             primary_metric_name="auc",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=100,
                             max_concurrent_runs=2)

## Submit the run

In [19]:
hyperdrive_run = experiment.submit(hd_config)

## Monitor

In [20]:
from azureml.widgets import RunDetails

RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Register the Model

In [18]:
print(run.get_file_names()[len(run.get_file_names())-1])

outputs/model.pickle


In [19]:
model = run.register_model(model_name='churn-model', 
                           model_path='outputs/model.pickle',
                           description='Churn Model',
                           tags={'Model': 'XGBoost', 'type': 'Classification'},
                          )